## 01. Trabalho Final
#### Dupla: Hélio Potelicki e Rennã Murilo Tiedt

In [1]:
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras import backend as K
from keras.layers import Dense
from tensorflow import keras
from numpy import argmax
import tensorflow as tf
from PIL import Image
import pandas as pd
import numpy as np
import os


K.set_image_data_format('channels_first')

%matplotlib inline

seed = 7
np.random.seed(seed)

In [2]:
dataset_path = "W:/CODANDO/FURB/FURB_AULAS/7 - Semestre/Processamento de Imagens/Trabalho Final/Classificador-de-figuras-musicais/database/"
df = pd.DataFrame(columns=["x_train", "y_train"])

for root, dirs, files in os.walk(dataset_path):
    for filename in files:
        if filename.endswith(".jpg"):
            image = np.asarray(Image.open((f"{root}/{filename}")))

            label = root.split("/")[-1]
            row = [image, label]
            df.loc[len(df)] = row

df = df.sample(frac=1)
df.reset_index(inplace=True)
df.drop(columns=["index"], inplace=True)
df.head()

,x_train,y_train
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",3
1,"[[[253, 253, 253], [253, 253, 253], [253, 253,...",0
2,"[[[253, 253, 253], [253, 253, 253], [253, 253,...",3
3,"[[[254, 254, 254], [254, 254, 254], [254, 254,...",4
4,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",3


In [3]:
# Train
x_train = df.iloc[:3000, 0:1].to_numpy()
x_train = np.array([img[0] for img in x_train])

y_train = df.iloc[:3000, 1:2]
y_train = tf.strings.to_number(y_train, out_type=tf.int32)
y_train = tf.one_hot(y_train, depth=5)
y_train = tf.squeeze(y_train)

# Test
x_test = df.iloc[3000:, 0:1].to_numpy()
x_test = np.array([img[0] for img in x_test])

y_test = df.iloc[3000:, 1:2]
y_test = tf.strings.to_number(y_test, out_type=tf.int32)
y_test = tf.one_hot(y_test, depth=5)
y_test = tf.squeeze(y_test)

In [4]:
print(x_train.dtype)
print(x_train.shape)

print(y_train.dtype)
print(y_train.shape)

print(x_test.dtype)
print(x_test.shape)

print(y_test.dtype)
print(y_test.shape)

uint8
(3000, 64, 64, 3)
<dtype: 'float32'>
(3000, 5)
uint8
(2000, 64, 64, 3)
<dtype: 'float32'>
(2000, 5)


In [5]:
# O model será exportado para este arquivo
filename='model.h5'

def model():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(3, 64, 64), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax', name='predict'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [6]:
model = model()
model.summary()

if not os.path.exists('./{}'.format(filename)):
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, batch_size=100)
    model.save(filename)
    model.save_weights(f"weights_{filename}")
else:
    model.load_weights('./{}'.format(filename))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 60, 60)        2280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 28, 28)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 14, 14)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 12, 12)        2040      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 6, 6)          0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 6, 6)          0

ValueError: in user code:

    C:\Users\Helio\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Helio\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Helio\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Helio\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Helio\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Helio

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("\nacc: %.2f%%" % (scores[1]*100))

In [ ]:
image_test = 'W:/CODANDO/FURB/FURB_AULAS/7 - Semestre/Processamento de Imagens/Trabalho Final/src/colcheia.png'

def load_image(img_path):
    from keras.preprocessing import image
    img = image.load_img(img_path, target_size=(64, 64))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.                                    

    return img_tensor

model = load_model("model_99.h5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

new_image = load_image(image_test)

pred = model.predict_classes(new_image)
pred_proba = model.predict_proba(new_image)
pred_proba = f"{(float(pred_proba[0][pred]) * 100):.2f}%"
fig_dict = {0:"Colcheia", 1:"Minima", 2:"Semibreve", 3:"Semicolcheia", 4:"Seminima"}

print(f"Figura de uma {fig_dict[pred[0]]}, com confiança de {pred_proba}")
im = Image.open(image_test)
im